In [1]:
# -*- coding: utf-8 -*-
import json
import math
 
x_pi = 3.14159265358979324 * 3000.0 / 180.0
pi = 3.1415926535897932384626  # π
a = 6378245.0  # 长半轴
ee = 0.00669342162296594323  # 扁率
 
def wgs84togcj02(lng, lat):
    """
    WGS84转GCJ02(火星坐标系)
    :param lng:WGS84坐标系的经度
    :param lat:WGS84坐标系的纬度
    :return:
    """
    if out_of_china(lng, lat):  # 判断是否在国内
        return lng, lat
    dlat = transformlat(lng - 105.0, lat - 35.0)
    dlng = transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [mglng, mglat]
 
 
def gcj02towgs84(lng, lat):
    """
    GCJ02(火星坐标系)转GPS84
    :param lng:火星坐标系的经度
    :param lat:火星坐标系纬度
    :return:
    """
    if out_of_china(lng, lat):
        return lng, lat
    dlat = transformlat(lng - 105.0, lat - 35.0)
    dlng = transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [lng * 2 - mglng, lat * 2 - mglat]
 
 
def transformlat(lng, lat):
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + \
        0.1 * lng * lat + 0.2 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * pi) + 40.0 *
            math.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * pi) + 320 *
            math.sin(lat * pi / 30.0)) * 2.0 / 3.0
    return ret
 
 
def transformlng(lng, lat):
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
        0.1 * lng * lat + 0.1 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * pi) + 40.0 *
            math.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * pi) + 300.0 *
            math.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    return ret
 
 
def out_of_china(lng, lat):
    """
    判断是否在国内，不在国内不做偏移
    :param lng:
    :param lat:
    :return:
    """
    if lng < 72.004 or lng > 137.8347:
        return True
    if lat < 0.8293 or lat > 55.8271:
        return True
    return False

### 全部店铺

In [2]:
import pandas as pd
allstores = pd.read_excel('汉堡数据＆汇总数据.xlsx', sheet_name='汇总')
allstores.head()

,店铺名,销量,评分,商圈,人均消费,店铺类型,大类,评论量,纬度,经度,行政区,详情页
0,DQ·蛋糕·冰淇淋（连城新天地店）,1000,4.4,市中心/会展中心,29,冰淇淋,饮品甜点,76,22.5350,114.0607,福田区,https://www.meituan.com/meishi/611279/
1,GODIVA（COCOPark店）,100,4.4,市中心/会展中心,64,冰淇淋,饮品甜点,32,22.5340,114.0536,福田区,https://www.meituan.com/meishi/82901426/
2,外婆家（卓悦汇店）,4000,4.3,梅林,59,杭帮菜,其他,1600,22.5695,114.0599,福田区,https://www.meituan.com/meishi/102380138/
3,新白鹿餐厅（京基KK ONE店）,100,4.2,上沙/下沙,68,杭帮菜,其他,38,22.5283,114.0263,福田区,https://www.meituan.com/meishi/157765161/
4,姜虎东白丁烤肉（卓悦汇店）,1,4.2,梅林,115,韩国菜,其他,1,22.5689,114.0599,福田区,https://www.meituan.com/meishi/185099560/


In [3]:
allstores = allstores.drop_duplicates()
latlons = allstores.loc[:, ['经度', '纬度']].values

In [4]:
newlat = []
newlon = []
for i in range(len(latlons)):
    [lng, lat] = latlons[i]
    [dstlng, dstlat] = gcj02towgs84(lng, lat)
    newlat.append(dstlat)
    newlon.append(dstlng)

In [5]:
edited_stores = allstores.loc[:, ['店铺名', '销量', '经度', '纬度']]
edited_stores.loc[:, '经度'] = newlon
edited_stores.loc[:, '纬度'] = newlat

In [6]:
edited_stores.to_excel('WGS84Stores.xlsx',index=False)

### BurgerPlaces

In [7]:
burger = pd.read_excel('汉堡数据＆汇总数据.xlsx', sheet_name='汉堡').drop_duplicates()
latlons = burger.loc[:, ['经度', '纬度']].values
newlat = []
newlon = []
for i in range(len(latlons)):
    [lng, lat] = latlons[i]
    [dstlng, dstlat] = gcj02towgs84(lng, lat)
    newlat.append(dstlat)
    newlon.append(dstlng)
edited_burger = burger.loc[:, ['店铺名', '销量', '经度', '纬度']]
edited_burger.loc[:, '经度'] = newlon
edited_burger.loc[:, '纬度'] = newlat
edited_burger.to_excel('WGS84Burger.xlsx',index=False)

### 房价

In [8]:
house = pd.read_csv('链家-深圳房价.csv', encoding = 'gbk').drop_duplicates()
house.head()

,标题,小区名字,地段,行政区,总价,单价（元/平）,经度,纬度,户型,面积,朝向,装修,楼层,时间,架构,详情
0,大阳台，高楼层视野无遮挡，安静宜居 楼下就是学 校,梅林一村三区,梅林,福田区,704.0,"77,900",114.040277,22.567783,3室2厅,90.29平米,南 北,精装,高楼层(共32层),2002年建,板塔结合,https://sz.lianjia.com/ershoufang/105112913772...
1,已改成三房 朝花园安静 诚心出售,明月花园,石厦,福田区,601.0,"72,600",114.055196,22.524461,3室2厅,82.66平米,西,精装,中楼层(共31层),1997年建,板楼,https://sz.lianjia.com/ershoufang/105112933259...
2,一区三房，户型好宜居家，中间位置楼层,园岭新村,园岭,福田区,780.0,"96,499",114.100529,22.554209,3室2厅,80.83平米,南,精装,高楼层(共6层),1983年建,板楼,https://sz.lianjia.com/ershoufang/105113101278...
3,福田高铁站旁低总价两房，满五唯一，自住看房方便！！,五洲星苑,景田,福田区,483.0,"77,400",114.045494,22.542072,2室1厅,62.4平米,北,简装,中楼层(共33层),2006年建,板塔结合,https://sz.lianjia.com/ershoufang/105113214811...
4,满五年，方正户型，南北通透，少有大户型，两个洗手间,文伟阁,梅林,福田区,1216.0,"74,174",114.043023,22.569288,4室2厅,163.94平米,南 北,简装,25层,2000年建,板塔结合,https://sz.lianjia.com/ershoufang/105108748308...


In [9]:
latlons = house.loc[:, ['经度', '纬度']].values
newlat = []
newlon = []
for i in range(len(latlons)):
    [lng, lat] = latlons[i]
    [dstlng, dstlat] = gcj02towgs84(lng, lat)
    newlat.append(dstlat)
    newlon.append(dstlng)
edited_house = house.loc[:, ['小区名字', '单价（元/平）', '经度', '纬度']]
edited_house.loc[:, '经度'] = newlon
edited_house.loc[:, '纬度'] = newlat

edited_house.columns = ['小区名字', 'Price', '经度', '纬度']

split = edited_house.Price.str.split(',')
lam = lambda x: x[0]+x[1]
newprice = []
for i in split:
    newprice.append(lam(i))
edited_house.loc[:, 'Price'] = newprice
edited_house.Price = edited_house.Price.astype('float')

In [10]:
import numpy as np
lower = np.quantile(edited_house.Price, 0.05)
upper = np.quantile(edited_house.Price, 0.95)
print(lower, upper)
edited_house = edited_house[edited_house.Price>lower]
edited_house = edited_house[edited_house.Price<upper]

56000.0 120000.0


In [11]:
edited_house.to_csv('WGS84House.csv', index=True)

### Office

In [12]:
office = pd.read_csv('RandomPOI.csv')
office.head()

,名称,经度,纬度
0,美林商业大厦,113.992878,22.536038
1,幸福阁,114.052415,22.523050
2,湾厦德园,113.929100,22.491432
3,向南海德商业大厦,113.920110,22.522480
4,科技联合大厦,113.944518,22.580612


In [13]:
latlons = office.loc[:, ['经度', '纬度']].values
newlat = []
newlon = []
for i in range(len(latlons)):
    [lng, lat] = latlons[i]
    [dstlng, dstlat] = gcj02towgs84(lng, lat)
    newlat.append(dstlat)
    newlon.append(dstlng)
edited_office = office.loc[:, ['名称', '经度', '纬度']]
edited_office.loc[:, '经度'] = newlon
edited_office.loc[:, '纬度'] = newlat
edited_office.to_excel('WGS84Office.xlsx',index=False)

### 塔斯汀

In [15]:
tst = pd.read_excel('塔斯汀.xls')
tst.head()

,店铺名,销量,评分,商圈,人均消费,店铺类型,大类,评论量,纬度,经度,行政区,详情页
0,塔斯汀中国汉堡（福田下沙店）,9999,5.0,上沙/下沙,17,西式快餐/汉堡,西式快餐,999,22.5247,114.0278,福田区,https://www.meituan.com/meishi/1656168138/
1,塔斯汀中国汉堡（南山动漫园店）,9999,4.7,南油,18,西式快餐/汉堡,西式快餐,72,22.5096,113.9240,南山区,https://www.meituan.com/meishi/798245982/
2,塔斯汀中国汉堡（科技路店）,9999,4.8,科技园,19,西式快餐/汉堡,西式快餐,60,22.5445,113.9501,南山区,https://www.meituan.com/meishi/1135781751/
3,塔斯汀中国汉堡（枫信科技园店）,9999,4.8,科技园,10,西式快餐/汉堡,西式快餐,17,22.5409,113.9388,南山区,https://www.meituan.com/meishi/649428485/
4,塔斯汀中国汉堡（西丽旺棠店）,8000,4.7,西丽,15,西式快餐/汉堡,西式快餐,158,22.5849,113.9483,南山区,https://www.meituan.com/meishi/1151131806/


In [16]:
latlons = tst.loc[:, ['经度', '纬度']].values
newlat = []
newlon = []
for i in range(len(latlons)):
    [lng, lat] = latlons[i]
    [dstlng, dstlat] = gcj02towgs84(lng, lat)
    newlat.append(dstlat)
    newlon.append(dstlng)
edited_tst = tst.loc[:, ['店铺名', '销量', '经度', '纬度']]
edited_tst.loc[:, '经度'] = newlon
edited_tst.loc[:, '纬度'] = newlat
edited_tst.to_excel('WGS84TST.xlsx',index=False)

### 全部汉堡店

In [17]:
ab = pd.read_excel('全部汉堡店数据.xls')
ab.head()

,店铺名,销量,评分,商圈,人均消费,店铺类型,大类,评论量,纬度,经度,行政区,详情页
0,塔斯汀中国汉堡（福田下沙店）,9999,5.0,上沙/下沙,17,西式快餐/汉堡,西式快餐,999,22.5247,114.0278,福田区,https://www.meituan.com/meishi/1656168138/
1,韦小堡?炸鸡汉堡（新洲店）,9999,4.9,新洲/石厦,22,西式快餐/汉堡,西式快餐,3898,22.5222,114.0463,福田区,https://www.meituan.com/meishi/1943439114/
2,塔斯汀中国汉堡（南山动漫园店）,9999,4.7,南油,18,西式快餐/汉堡,西式快餐,72,22.5096,113.9240,南山区,https://www.meituan.com/meishi/798245982/
3,塔斯汀中国汉堡（科技路店）,9999,4.8,科技园,19,西式快餐/汉堡,西式快餐,60,22.5445,113.9501,南山区,https://www.meituan.com/meishi/1135781751/
4,塔斯汀中国汉堡（枫信科技园店）,9999,4.8,科技园,10,西式快餐/汉堡,西式快餐,17,22.5409,113.9388,南山区,https://www.meituan.com/meishi/649428485/


In [18]:
latlons = ab.loc[:, ['经度', '纬度']].values
newlat = []
newlon = []
for i in range(len(latlons)):
    [lng, lat] = latlons[i]
    [dstlng, dstlat] = gcj02towgs84(lng, lat)
    newlat.append(dstlat)
    newlon.append(dstlng)
edited_ab = ab.loc[:, ['店铺名', '销量', '经度', '纬度']]
edited_ab.loc[:, '经度'] = newlon
edited_ab.loc[:, '纬度'] = newlat
edited_ab.to_excel('WGS84AllBurger.xlsx',index=False)